In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import re
import shutil
import time
import zipfile

from pathlib import Path
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import secrets
import util

def main():
    chrome_options = webdriver.ChromeOptions()
    
    prefs = {
        'download.default_directory': secrets.acorn_download_dir,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True
    }
    chrome_options.add_experimental_option('prefs', prefs)
    
    driver = webdriver.Chrome(options=chrome_options)

    wait = WebDriverWait(driver, 10)

    driver.get(secrets.acorn_url)

    # Get rid of the cookies popup
    iframe_element = wait.until(EC.element_to_be_clickable((By.ID, "consent-iframe")))
    driver.switch_to.frame(iframe_element)

    iframe_element2 = wait.until(EC.element_to_be_clickable((By.ID, "ccpa-iframe")))
    driver.switch_to.frame(iframe_element2)
    print(f'{iframe_element2=}')

    accept_button = wait.until(EC.element_to_be_clickable((By.ID, 'accept_all_cookies_button')))
    accept_button.click()

    driver.switch_to.default_content()
    
    year_folder = wait.until(EC.element_to_be_clickable((By.XPATH, f"//div[contains(text(), '{secrets.year}')]")))
    year_folder.click()
    
    download_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@aria-label='Download']")))
    download_button.click()
    time.sleep(5)
    print('Done!')

    zip_file_path = Path(secrets.acorn_download_dir) / f'{secrets.year}.zip'
    extract_to_directory = secrets.acorn_download_dir

    time.sleep(2)

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_directory)

    util.move_from_download_to_staging(secrets.acorn_input_regex, secrets.acorn_output_regex,
                                       secrets.acorn_download_dir, 
                                       secrets.acorn_staging_dir,
                                       secrets.acorn_account_dirs, 'Acorn',
                                       secrets.acorn_allow_patterns)
    util.copy_to_destination(secrets.acorn_account_dirs,
                            secrets.acorn_staging_dir,
                            secrets.destination_dir)

    # cleanup
    shutil.rmtree(secrets.acorn_staging_dir)
    shutil.rmtree(secrets.acorn_download_dir)


main()

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000100fe6a80 chromedriver + 4385408
1   chromedriver                        0x0000000100fdf38c chromedriver + 4354956
2   chromedriver                        0x0000000100bfcb0c chromedriver + 281356
3   chromedriver                        0x0000000100c3f2f8 chromedriver + 553720
4   chromedriver                        0x0000000100c77d24 chromedriver + 785700
5   chromedriver                        0x0000000100c33eec chromedriver + 507628
6   chromedriver                        0x0000000100c348c4 chromedriver + 510148
7   chromedriver                        0x0000000100fae43c chromedriver + 4154428
8   chromedriver                        0x0000000100fb2ea0 chromedriver + 4173472
9   chromedriver                        0x0000000100f93ff8 chromedriver + 4046840
10  chromedriver                        0x0000000100fb378c chromedriver + 4175756
11  chromedriver                        0x0000000100f86fb8 chro